In [ ]:
import yaml
import time
from datetime import datetime
from megadetector.detection import run_detector
import pandas as pd

import sys
sys.path.append('../scripts')  # Adjust the path as needed to point to the directory containing utils.py

from alert_system_utils import detector, classifier, set_device, check_emails, extract_and_update_camera_info, update_camera_data_dataframe, batch_classification, current_time, generate_alert_caption, check_counts_and_species

In [ ]:
# Load settings from configuration file
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

IMAP_HOST = config['imap_config']['host']
EMAIL_USER = config['imap_config']['user']
EMAIL_PASS = config['imap_config']['password']
TELEGRAM_BOT_TOKEN = config['telegram_config']['bot_token']
TELEGRAM_CHAT_ID = '-1002249589791' # replace with config after tests

# Detection and Classification Model Settings
DETECTOR_MODEL_PATH = '../models/md_v5a.0.0.pt'
DETECTOR_CLASSES = ["animal", "human", "vehicle"]
DETECTION_THRESHOLD = 0.10

BACKBONE = 'vit_large_patch14_dinov2'
CLASSIFIER_MODEL_PATH = '../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt'
CLASSIFIER_CLASSES = ["Badger", "Ibex", "Red Deer", "Chamois", "Cat", "Goat", "Roe Deer", "Dog", "Squirrel", "Equid", "Genet", "Hedgehog", "Lagomorph", "Wolf", "Lynx", "Marmot", "Micromammal", "Mouflon", "Sheep", "Mustelid", "Bird", "Bear", "Nutria", "Fox", "Wild Boar", "Cow"]
SPECIES_OF_INTEREST = ["Wild Boar", "Bear"]
CLASSIFICATION_THRESHOLD = 0.20

CAPTURE_DATABASE_PATH = '../data/capture_database.csv'
CAMERA_LOCATIONS_PATH = '../data/camera_locations.csv'
ALERT_LANGUAGE = "en" # Enter 'en' for English or 'ro' for Romanian

# Initialise the Detection and Classifier Models
device = set_device()
detector_model = run_detector.load_detector(DETECTOR_MODEL_PATH)
print("Loading classifier...")
start_time = time.time()
classifier_model = classifier(CLASSIFIER_MODEL_PATH, BACKBONE, CLASSIFIER_CLASSES, device)
end_time = time.time()
print(f"Loaded classifier in {(end_time - start_time):.2f} seconds")

In [ ]:
if __name__ == "__main__":
    print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
    while True:
        try:
            images, camera_id, temp_deg_c, img_date, img_time, battery, sd_memory = \
                check_emails(IMAP_HOST, EMAIL_USER, EMAIL_PASS)
            
            if camera_id:

                camera_make, gps, location, map_url, battery, sd_memory = extract_and_update_camera_info(CAMERA_LOCATIONS_PATH, camera_id, battery, sd_memory)
                
                # TO DO: if camera_id is UOVision, wait 10, check for another, then append or continue.
                # check subject to determine if uovision, then only open if it is

                df = pd.read_csv(CAPTURE_DATABASE_PATH)
                df = update_camera_data_dataframe(df, len(images), camera_id, camera_make, img_date, img_time, temp_deg_c, battery, sd_memory, location, gps, map_url)

                if images:

                    df = detector(df, detector_model, images, DETECTION_THRESHOLD)
                    
                    df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD) # test classification threshold

                    if check_counts_and_species(df, images):

                        df, alert_caption, human_warning, priority_alert = generate_alert_caption(df, len(images), SPECIES_OF_INTEREST, EMAIL_USER, ALERT_LANGUAGE)

                        # alert_images = annotate_images(df, images)

                        # send_alert(alert_caption, alert_images)

                        # print(f"\n\n Test Alert Caption: \n\n {alert_caption}")

                        # df = save_images(df, images)
                
                df.to_csv(CAPTURE_DATABASE_PATH, index=False)
                print(f"{current_time()} | Capture Database Updated: {CAPTURE_DATABASE_PATH}")
                #del df
                
                # delete email

            else:
                time.sleep(1)

        except KeyboardInterrupt:
            print(f"{current_time()} | Interrupted by user")
            break

        # except Exception as e:
        #    print(f"An error occurred: {e}")
        #    print(f"\nMonitoring {EMAIL_USER} for new messages...")
        #    continue


In [ ]:
def annotate_images(df, images):

    # Animal - red
    # Human - green
    # Vehicle - blue

    return df, alert_images

In [ ]:
def send_alert(alert_caption, alert_images):

    print(f"{current_time()} | Alert sent")

In [ ]:
def save_images(df, images):

    return df